
# 第7课：时序差分方法（Temporal-Difference Methods）

> non-incremental to incremental

---

## Part 1：例子 —— 为什么只用当前一步也能学值？

---

### ✅ 回顾：Monte Carlo（MC）方法的问题

MC 方法需要等一个**完整 episode** 结束后，才能计算真实的 return：

$$
G_t = r_{t+1} + \gamma r_{t+2} + \gamma^2 r_{t+3} + \cdots
$$

这种做法的问题：

* **延迟反馈**：要等到 episode 结束；
* **高方差**：因为 return $G_t$ 是一条完整轨迹的累计值，波动大；
* **不能处理 continuing task**：MC 不适合无限步的任务。

---

### TD 方法提出的核心思想

> **“只使用一步奖励 + 当前的估计值函数” 来更新值函数，不用等完整轨迹”**

具体例子：

假设你在玩游戏，只玩到第 1 步 $s_0 \to s_1$，就提前终止了。MC 不能学习，但：

* 你收到了 $r_1$；
* 你知道 $s_1$ 的估计值 $V(s_1)$；
* 可以组合形成新的估计值：

$$
V(s_0) \leftarrow V(s_0) + \alpha \left[ r_1 + \gamma V(s_1) - V(s_0) \right]
$$

这就是 **Temporal-Difference Learning** 的核心思想。






## Part 2：TD算法介绍（TD(0)）

---

### TD(0) 算法定义

**TD(0)** 是最基础的 TD 方法，它的目标是估计策略 $\pi$ 下的状态值函数 $V_\pi(s)$，公式如下：

$$
v(s_t) \leftarrow v(s_t) + \alpha \left[ r_{t+1} + \gamma v(s_{t+1}) - v(s_t) \right]
$$

这一更新被称为 **TD target** 和 **TD error**：

* TD 目标（target）：

  $$
  r_{t+1} + \gamma v(s_{t+1})
  $$
* TD 误差（error）：

  $$
  \delta_t = r_{t+1} + \gamma v(s_{t+1}) - v(s_t)
  $$

---

### TD(0) 伪代码

```python
Initialize V(s) arbitrarily
for each episode:
    Initialize s
    while s is not terminal:
        Take action a under policy π, observe r, s'
        V(s) ← V(s) + α * (r + γ * V(s') - V(s))
        s ← s'
```

---

### 特点总结

| 特点                   | 描述                       |
| -------------------- | ------------------------ |
| 在线学习                 | 可在每一步就更新，不需等 episode 结束  |
| 引入 bootstrapping     | 当前估计依赖未来估计值 $V(s_{t+1})$ |
| 高效                   | 比 MC 更快、稳定性好             |
| 适用于 continuing tasks | 因为不依赖完整 return           |
| 可能有偏                 | 因为使用了估计值而不是真实 return     |



## Part 3：TD算法收敛性与 MC 的比较

---

### TD 与 MC 的收敛对比

| 比较项                    | TD(0)          | Monte Carlo  |
| ---------------------- | -------------- | ------------ |
| 是否需要完整 episode         | ❌ 否            | ✅ 是          |
| 是否 bootstrapping       | ✅ 是（使用下一状态的估计） | ❌ 否（用真实 G 值） |
| 方差                     | ✅ 小（低方差）       | ❌ 大（高方差）     |
| 偏差                     | ❌ 有偏（因用估计值）    | ✅ 无偏（使用实际 G） |
| 收敛速度                   | ✅ 快            | ❌ 慢          |
| 是否适用于 continuing tasks | ✅ 是            | ❌ 否          |

---

### TD 的收敛理论（简述）

在满足如下条件时，**TD(0) 可以保证收敛到 $v^\pi$**：

* 使用固定策略 $\pi$（即 **策略评估**）；
* 所有状态足够多次被访问；
* 步长 $\alpha_t$ 满足 Robbins-Monro 条件：

  $$
  \sum \alpha_t = \infty, \quad \sum \alpha_t^2 < \infty
  $$

这些条件保证了即使使用带噪的 bootstrapped target，TD(0) 也能收敛。

---

### 直觉理解：

* **MC**：像是“每次玩完整个游戏，记录结果，再更新策略”；
* **TD**：像是“边玩边学，每走一步都更新一次策略”。

---

### 总结：MC vs TD 的权衡

| 优势   | MC            | TD(0)        |
| ---- | ------------- | ------------ |
| 收敛结果 | 无偏估计          | 有偏估计（但可以更快）  |
| 方差   | 高             | 低            |
| 实用性  | 离线、episode型任务 | 在线、长期任务      |
| 更新时机 | 等 episode 结束  | 每一步都可更新      |
| 收敛效率 | 慢             | 快（特别是在实际应用中） |





## Part 4：SARSA 算法（On-policy）

---

### 什么是 SARSA？

SARSA 是一个**on-policy** TD 控制方法，用于在与环境交互的过程中学习最优动作值函数 $Q_\pi(s,a)$。

**名称来源**于其更新用到的五元组：

$$
\text{SARSA} = (s_t, a_t, r_{t+1}, s_{t+1}, a_{t+1})
$$

### 更新公式

$$
Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha \left[ r_{t+1} + \gamma Q(s_{t+1}, a_{t+1}) - Q(s_t, a_t) \right]
$$

* 使用当前策略选择下一个动作 $a_{t+1}$，并用它的 Q 值做 bootstrapping；
* 典型的 **on-policy**：策略更新和采样使用同一个策略。

---

### SARSA 伪代码（on-policy TD 控制）

![](./img/7_1.png)


```python
Initialize Q(s,a) arbitrarily
for each episode:
    Initialize s
    Choose a from s using ε-greedy policy
    repeat:
        Take action a, observe r and s'
        Choose a' from s' using ε-greedy policy
        Q(s,a) ← Q(s,a) + α [r + γ * Q(s',a') - Q(s,a)]
        s ← s'
        a ← a'
    until s is terminal
```

---

### 特点

| 特点            | 描述                                        |
| ------------- | ----------------------------------------- |
| 策略类型          | On-policy（更新和行为使用同一个策略）                   |
| Bootstrapping | 是（使用估计值 $Q(s', a')$）                      |
| 探索策略          | 通常为 ε-greedy                              |
| 收敛性           | 对于 GLIE（greedy in the limit）策略，SARSA 保证收敛 |
| 应用场景          | 环境变化快、策略安全性重要（如机器人控制）                     |






## Part 5：Expected SARSA 与 n-step SARSA

---

### 5.1 Expected SARSA（期望 SARSA）

---

#### 核心思想

* SARSA 使用下一个动作 $a_{t+1}$ 的 Q 值作为 target；
* **Expected SARSA** 改为使用 **下一个状态中所有动作的期望 Q 值**，即对 $a' \sim \pi(a'|s')$ 做期望。

---

#### 更新公式：

$$
Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha \left[ r_{t+1} + \gamma \sum_{a'} \pi(a'|s_{t+1}) Q(s_{t+1}, a') - Q(s_t, a_t) \right]
$$

这更稳定，因为移除了 $a_{t+1}$ 的采样噪声。

---

#### 特点：

| 项目        | SARSA                        | Expected SARSA |
| --------- | ---------------------------- | -------------- |
| 下一个动作     | 采样一个动作 $a_{t+1}$             | 对所有动作求加权平均     |
| target 方差 | 高                            | 低              |
| 计算量       | 低                            | 稍高（要遍历所有 $a'$） |
| 收敛速度      | 慢                            | 稳定、快           |
| 应用        | 非常适合带有策略网络的 Actor-Critic 结构等 |                |

---

### 5.2 n-step SARSA

---

#### 动机：信息利用更广泛

* TD(0) 只利用一步信息；
* n-step SARSA 折中 MC 与 TD(0)，用前 n 步的回报来更新 $Q(s_t, a_t)$。

---

#### 更新公式：

$$
Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha \left[ G^{(n)}_t - Q(s_t, a_t) \right]
$$

其中：

$$
G^{(n)}_t = r_{t+1} + \gamma r_{t+2} + \cdots + \gamma^{n-1} r_{t+n} + \gamma^n Q(s_{t+n}, a_{t+n})
$$

* 这叫做 **n-step return**；
* 它兼具 MC 的完整回报和 TD 的 bootstrapping；
* $n=1$ 时等价于 TD(0)；
* $n \to \infty$ 时接近 MC。

---

### 小结：SARSA vs Expected SARSA vs n-step

| 方法             | 核心思想                        | variance | bias | bootstrapping | 计算复杂度    |
| -------------- | --------------------------- | -------- | ---- | ------------- | -------- |
| SARSA          | 使用实际采样的下一动作                 | 高        | 中    | 是             | 低        |
| Expected SARSA | 对下一个状态下所有动作取期望              | 低        | 更低   | 是             | 中        |
| n-step SARSA   | 使用多步回报 + 最后一项 bootstrapping | 介于两者之间   | 低\~中 | 是             | 高（取决于 n） |







## Part 6：Q-learning 介绍（Off-policy）

---

### 什么是 Q-learning？

Q-learning 是一种非常重要的**off-policy TD 控制算法**，目标是学习**最优动作值函数** $Q_*(s, a)$，从而导出**最优策略**：

$$
\pi^*(s) = \arg\max_a Q_*(s, a)
$$

---

### 更新公式（核心）

$$
Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha \left[ r_{t+1} + \gamma \max_{a'} Q(s_{t+1}, a') - Q(s_t, a_t) \right]
$$

与 SARSA 不同之处在于：

* SARSA：使用当前策略选择的动作 $a_{t+1}$；
* Q-learning：**直接使用 max，假设以后都采取最优动作**；
* 因此 Q-learning 是 **off-policy**：学习目标与行为策略不一致。

---

### 特点总结

| 特性   | 描述                              |
| ---- | ------------------------------- |
| 策略类型 | **Off-policy**（行为策略 ≠ 目标策略）     |
| 更新目标 | 使用最大值 $\max_{a'} Q(s', a')$     |
| 方差   | 较大（使用最大值带来不稳定性）                 |
| 收敛性  | 若每对 $(s, a)$ 被无限次访问，学习率满足条件，则收敛 |
| 实用性  | 广泛应用于 Atari、导航、离线RL 等问题中        |

---

## Part 7：Q-learning 伪代码与例子

![](./img/7_2.png)

![](./img/7_3.png)

---

### 伪代码（Q-learning）

```python
Initialize Q(s,a) arbitrarily
for each episode:
    Initialize s
    repeat:
        Choose a from s using ε-greedy(Q)
        Take action a, observe r and s'
        Q(s,a) ← Q(s,a) + α [r + γ * max_a' Q(s',a') - Q(s,a)]
        s ← s'
    until s is terminal
```

* 使用 ε-greedy 策略选择动作（探索）；
* 用最大 Q 值作为更新目标（学习最优策略）；
* 只需一个状态转移，即可进行更新（TD 方法的优势）。

---

### 示例：

考虑如下简单迷宫：

```
S → A → G
```

其中：

* S 为起点，G 为目标；
* A 只有两个动作：左（回到 S）和右（到 G）；
* 到 G 得 +10 奖励，其它为 0；
* 假设 ε=0.1，α=0.5，γ=0.9，初始 Q 值为 0。

在第一次执行 S→A→G 后，更新：

$$
Q(A, \text{right}) \leftarrow 0 + 0.5 \cdot (10 + 0 - 0) = 5
$$

随着不断交互，Q(A, right) 会收敛到接近真实期望 9。






## Part 8：TD算法统一形式与总结

---

### 统一 TD 更新形式

我们可以将所有 TD 控制类算法的更新公式统一写为：

$$
Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha \left[ \underbrace{\text{Target}_t}_{\text{TD目标}} - Q(s_t, a_t) \right]
$$

其中 TD 目标的不同定义如下：

| 算法                 | TD目标（Target）                                | 策略类型                 |           |
| ------------------ | ------------------------------------------- | -------------------- | --------- |
| **SARSA**          | $r_{t+1} + \gamma Q(s_{t+1}, a_{t+1})$      | On-policy            |           |
| **Expected SARSA** | ( r\_{t+1} + \gamma \sum\_{a'} \pi(a'       | s\_{t+1}) Q(s',a') ) | On-policy |
| **Q-learning**     | $r_{t+1} + \gamma \max_{a'} Q(s_{t+1}, a')$ | Off-policy           |           |

---

### 比较与总结

| 方法             | 策略类型       | 是否 bootstrapping | 是否使用 max  | 是否使用策略 π | 目标收敛性       |
| -------------- | ---------- | ---------------- | --------- | -------- | ----------- |
| SARSA          | On-policy  | ✅ 是              | ❌ 否       | ✅ 是      | 收敛于 $Q^\pi$ |
| Expected SARSA | On-policy  | ✅ 是              | ❌ 否（但更稳定） | ✅ 是      | 收敛于 $Q^\pi$ |
| Q-learning     | Off-policy | ✅ 是              | ✅ 是       | ❌ 否      | 收敛于 $Q^*$   |

---

### 理解总结：

* **TD 方法 = 及时更新 + bootstrapping**；
* TD 比 MC 更高效，能在 episode 未结束时学习；
* 控制方法（如 SARSA、Q-learning）通过估计 $Q$ 函数引导策略改进；
* Q-learning 是学习最优策略的强大武器，但可能收敛慢、振荡；
* Expected SARSA 折中偏差和方差，更稳定但计算量大。


